# DINO ViT-S/16 Feature Extraction on CIFAR-100

## Compute feature vectors

In [1]:
import torch
import torch.nn as nn
from typing import cast
from torchvision import transforms
from torchvision.datasets import CIFAR100
import os

# Load DINO ViT-S/16 pre-trained from torch.hub

dino_model = cast(
    nn.Module,
    torch.hub.load("facebookresearch/dino:main", "dino_vits16", pretrained=True),
)
dino_model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dino_model.to(device=device)

# Use the preprocess defined in the previous cell
# Make sure the dataset uses the correct preprocess
preprocess = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        # in federated training, we should consider to use mean and std of the cifar100
        # these are the parameters on which dino was trained
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

train_dataset = CIFAR100(root="./data", train=True, download=True, transform=preprocess)
test_dataset = CIFAR100(root="./data", train=False, download=True, transform=preprocess)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)


# Function to extract features from a dataloader
def extract_features_and_labels(dataloader, model, device):
    all_features = []
    all_labels = []
    with torch.no_grad():
        total_batches = len(dataloader)
        for batch_idx, (images, labels) in enumerate(dataloader):
            images = images.to(device)
            # Get features from the backbone (without the classification head)
            features = model(images)
            all_features.append(features.cpu())
            all_labels.append(labels.cpu())
            if (batch_idx + 1) % 10 == 0 or (batch_idx + 1) == total_batches:
                print(
                    f"Batch {batch_idx + 1}/{total_batches} ({(batch_idx + 1) / total_batches:.1%}) completed"
                )
    all_features = torch.cat(all_features, dim=0)
    all_labels = torch.cat(all_labels, dim=0)
    return all_features, all_labels


#Create the directory if it doesn't exist
os.makedirs("features", exist_ok=True)

# Extract features and labels for train
train_features, train_labels = extract_features_and_labels(
    train_loader, dino_model, device
)
torch.save(
    {"features": train_features, "labels": train_labels},
    "features/train_features.pt",
)

# Extract features and labels for test
test_features, test_labels = extract_features_and_labels(
    test_loader, dino_model, device
)
torch.save(
    {"features": test_features, "labels": test_labels}, "features/test_features.pt"
)

Downloading: "https://github.com/facebookresearch/dino/zipball/main" to /root/.cache/torch/hub/main.zip
Downloading: "https://dl.fbaipublicfiles.com/dino/dino_deitsmall16_pretrain/dino_deitsmall16_pretrain.pth" to /root/.cache/torch/hub/checkpoints/dino_deitsmall16_pretrain.pth


100%|██████████| 82.7M/82.7M [00:00<00:00, 450MB/s]
100%|██████████| 169M/169M [00:03<00:00, 47.7MB/s]


Batch 10/1563 (0.6%) completed
Batch 20/1563 (1.3%) completed
Batch 30/1563 (1.9%) completed
Batch 40/1563 (2.6%) completed
Batch 50/1563 (3.2%) completed
Batch 60/1563 (3.8%) completed
Batch 70/1563 (4.5%) completed
Batch 80/1563 (5.1%) completed
Batch 90/1563 (5.8%) completed
Batch 100/1563 (6.4%) completed
Batch 110/1563 (7.0%) completed
Batch 120/1563 (7.7%) completed
Batch 130/1563 (8.3%) completed
Batch 140/1563 (9.0%) completed
Batch 150/1563 (9.6%) completed
Batch 160/1563 (10.2%) completed
Batch 170/1563 (10.9%) completed
Batch 180/1563 (11.5%) completed
Batch 190/1563 (12.2%) completed
Batch 200/1563 (12.8%) completed
Batch 210/1563 (13.4%) completed
Batch 220/1563 (14.1%) completed
Batch 230/1563 (14.7%) completed
Batch 240/1563 (15.4%) completed
Batch 250/1563 (16.0%) completed
Batch 260/1563 (16.6%) completed
Batch 270/1563 (17.3%) completed
Batch 280/1563 (17.9%) completed
Batch 290/1563 (18.6%) completed
Batch 300/1563 (19.2%) completed
Batch 310/1563 (19.8%) completed
B

## Custom Dino

In [2]:
from typing import Optional, cast
from torch import nn
import torch
from pathlib import Path

class CustomDino(nn.Module):
    def __init__(self, num_classes: int = 100, backbone: Optional[nn.Module] = None, frozen_head: nn.Module = None):
        super().__init__()
        if backbone is None:
            backbone = cast(
                nn.Module,
                torch.hub.load(
                    "facebookresearch/dino:main", "dino_vits16", pretrained=True
                ),
            )
        self.backbone: nn.Module = backbone
        self.num_classes = num_classes

        # Attach the head
        if frozen_head is not None:
            self.head = frozen_head
        else:
            self.head = nn.Linear(384, num_classes)

        # Add a learnable or fixed scale factor (Temperature)
        # 30.0 is a standard value for Cosine Classifiers (FaceNet, etc.)
        self.scale = nn.Parameter(torch.tensor(30.0))

    def forward(self, x: torch.Tensor):
        features = self.backbone(x)

        # CRITICAL FIX 1: Normalize features so the Linear layer acts as Cosine Similarity
        features = torch.nn.functional.normalize(features, p=2, dim=1)

        logits = self.head(features)

        # CRITICAL FIX 2: Apply Temperature Scaling
        # This expands the range from [-1, 1] to [-30, 30], allowing Softmax to work
        return logits * self.scale


## Compute Centroids function


In [3]:
def compute_centroid(features, m,feature_size,DEVICE):
    features = features.to(DEVICE)
    # Compute mean of the class
    with torch.no_grad():
        features = F.normalize(features, p=2, dim=1)
        class_mean = torch.mean(features, dim=0)

        # Herding Selection
        exemplar_features = []

        # We assume features are [N, D]
        # We iterate m times to pick m samples
        for k in range(m):
            S = (
                torch.sum(torch.stack(exemplar_features), dim=0)
                if len(exemplar_features) > 0
                else torch.zeros(feature_size).to(DEVICE)
            )

            # Objective: minimize || class_mean - (S + phi(x)) / k   ||
            phi = features  # [N, D]
            mu = class_mean  # [D]
            # Distance for all candidates
            dists = torch.norm(mu - ((S + phi) / k), dim=1)
            # Pick best that isn't already chosen (simple way: set dist to inf)
            # In strict implementation, we remove the index.
            best_idx = torch.argmin(dists).item()
            exemplar_features.append(features[best_idx])

            # Mask this index so it's not picked again
            features[best_idx] = features[best_idx] + 10000  # Hacky mask
    return torch.mean(torch.stack(exemplar_features), dim=0)

In [5]:
!pip install wandb --quiet
import wandb

# Opzionale: fai il login subito se non l'hai già fatto
wandb.login()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: daviderandino (daviderandino-politecnico-di-torino) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [6]:
from torch.nn import Module

def save_checkpoint_to_wandb(
    run: wandb.Run,
    checkpoint: dict,
    filename: str = "model.pth",
    metadata: dict = None,
) -> None:
    """Save a PyTorch model to WandB as an artifact."""

    # 1. Save model locally
    torch.save(checkpoint, filename)

    # 2. Create artifact
    artifact_name = f"{run.group}-checkpoints" if hasattr(run, "group") and run.group else "checkpoints"
    artifact = wandb.Artifact(
        name=artifact_name,
        type="model",
        metadata=metadata or {}
    )

    artifact.add_file(filename)

    # 3. Log artifact to the existing run
    run.log_artifact(artifact)

    print(f"Model saved to WandB as artifact '{artifact_name}'.")


def load_checkpoint_from_wandb(
    run: wandb.Run,
    model: Module,
    filename: str = "model.pth",
    version: str = "latest"
) -> tuple[dict, wandb.Artifact] | None:
    """Download the latest model artifact and load it into `model`."""
    try:
        artifact_name = f"{run.group}-checkpoints" if hasattr(run, "group") and run.group else "checkpoints"
        artifact = run.use_artifact(f"{artifact_name}:{version}", type="model")
        artifact_dir = artifact.download()
        model_path = Path(artifact_dir) / filename
        print(f"Loading model from: {model_path}")
        checkpoint = torch.load(model_path, model.device if hasattr(model, 'device') else None, weights_only=False)
        print(f"Successfully loaded model from: {model_path}")
        return checkpoint, artifact
    except Exception as e :
        print(e)
        print(f"Model checkpoint not found on WandB. {e}")


## Train

In [11]:
ENTITY = "aml-fl-project"
PROJECT = "fl-task-arithmetic"
GROUP = "baseline-Davide-collab-test"

BATCH_SIZE = 64
LR  = 10e-4          # Learning Rate (Tune this: 0.1, 0.01, 0.001)
MOMENTUM = 0.9
WEIGHT_DECAY = 5e-4
EPOCHS = 10
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
PATIENCE = 3

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import Compose, Normalize, ToTensor, Resize, CenterCrop
from tqdm import tqdm
import wandb


# Standard CIFAR-100 Normalization
stats = ((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))

# Transforms
transform_train = Compose([
    Resize(256), CenterCrop(224), # Required for DINO
    # transforms.RandomHorizontalFlip(), # Optional augmentation
    ToTensor(),
    Normalize(*stats),
])

transform_test = Compose([
    Resize(256), CenterCrop(224),
    ToTensor(),
    Normalize(*stats),
])

# 1. LOAD RAW IMAGES (Required for fine-tuning backbone)
# We use the raw dataset, not the precomputed features, so we can backprop through the last layer.
train_dataset = datasets.CIFAR100(root="./data", train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR100(root="./data", train=False, download=True, transform=transform_test)

trainloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
testloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# 2. CALCULATE CENTROIDS (To initialize classifier)
print("Calculating Centroids for initialization...")
train_data = torch.load("features/train_features.pt")
saved_features, saved_labels = train_data["features"], train_data["labels"]

num_classes = 100
# Assuming features are dim 384 for ViT-S
feature_dim = saved_features.shape[1]
centroids = torch.zeros((num_classes, feature_dim))

for c in range(num_classes):
    # Select features belonging to class c
    class_features = saved_features[saved_labels == c]
    if len(class_features) > 0:
        # I computed the centroids with the paper herding method, but you can also just do the mean
        # centroids[c] = torch.mean(class_features, dim=0)
        centroids[c] = compute_centroid(class_features, class_features.shape[0],feature_dim,DEVICE)

# Normalize centroids (DINO features are often cosine-similar)
centroids = F.normalize(centroids, p=2, dim=1)
print("Centroids calculated.")

# Construct the fixed linear layer
classifier_weights = centroids # If it's already a tensor

LinearLayer = nn.Linear(384, 100)

# Load the weights
with torch.no_grad():
    LinearLayer.weight.copy_(classifier_weights)
    LinearLayer.bias.zero_()

# DO NOT FREEZE THE LAYER
# Allow the centroids to adjust as the backbone changes
# for param in LinearLayer.parameters():
#    param.requires_grad = False  <-- COMMENT THIS OUT

def train(lr, momentum, weight_decay, epochs, centroids):
    best_accuracy = 0.0
    patience_counter = 0
    run_id = f"run-222-centralized-dino-icarl-cifar100-lr{lr}-mom{momentum}-wd{weight_decay}"
    run = wandb.init(
        entity=ENTITY,
        project=PROJECT,
        group=GROUP,
        name=f"centralized-dino-icarl-cifar100-lr{lr}-mom{momentum}-wd{weight_decay}",
        id=run_id,
        resume="allow",
        mode="online",
    )

    # Initialize model with the frozen centroid head
    model = CustomDino(num_classes=100, frozen_head=LinearLayer).to(DEVICE)

    # Load checkpoint (if any)
    checkpoint = load_checkpoint_from_wandb(run, model, "model.pth")
    start_epoch = 0
    if checkpoint is not None:
        checkpoint_dict, artifact = checkpoint
        model.load_state_dict(checkpoint_dict['model'])
        start_epoch = artifact.metadata["epoch"] + 1
        print(f"Resuming from epoch {start_epoch}")
    else:
        print("Starting from scratch")

    criterion = nn.CrossEntropyLoss()
    # CRITICAL FIX 3: Add the HEAD parameters to the optimizer
    # We use a slightly higher LR for the head usually, but same LR is fine for now
    optimizer = optim.SGD(
        list(model.backbone.parameters()) + list(model.head.parameters()) + [model.scale],
        lr=lr,
        momentum=momentum,
        weight_decay=weight_decay
    )
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

    # We don't need centroids in the loop anymore; they are embedded in the model.head

    for epoch in range(start_epoch, epochs):
        # --- TRAINING ---
        model.train()
        running_loss = 0.0

        pbar = tqdm(trainloader, desc=f"Epoch {epoch+1}/{EPOCHS}")
        for images, labels in pbar:
            images, labels = images.to(DEVICE), labels.to(DEVICE)

            optimizer.zero_grad()

            # Model returns Logits (scores), not features
            outputs = model(images)

            # CrossEntropy expects (Logits, Class_Indices)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            pbar.set_postfix({'loss': loss.item()})

        scheduler.step()
        avg_train_loss = running_loss / len(trainloader)

        # --- EVALUATION ---
        model.eval()
        test_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for images, labels in testloader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)

                # 1. Forward Pass
                # The Frozen Linear Layer calculates the similarity to centroids for us.
                outputs = model(images) # Shape: [Batch_Size, 100]

                # 2. Loss
                loss = criterion(outputs, labels)
                test_loss += loss.item()

                # 3. Accuracy
                # The class with the highest score (dot product) is the nearest centroid.
                # No need for manual torch.cdist calculation.
                _, predicted = outputs.max(1)

                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()

        avg_test_loss = test_loss / len(testloader)
        acc = 100. * correct / total

        print(f"Epoch {epoch+1} Results: Train Loss: {avg_train_loss:.4f} | Test Loss: {avg_test_loss:.4f} | Test Acc: {acc:.2f}%")

        # Checkpointing
        if acc > best_accuracy:
            best_accuracy = acc
            save_checkpoint_to_wandb(run, {
                'model': model.state_dict(),
            }, "model.pth", {
                "task": model,
                "accuracy": acc,
                "epoch": epoch
            })
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter > PATIENCE:
                print("Early stopping triggered.")
                break

        print(epoch, "Saved checkpoint model to WandB.")

train(
    lr=LR,
    momentum=MOMENTUM,
    weight_decay=WEIGHT_DECAY,
    epochs=EPOCHS,
    centroids=centroids
)

Calculating Centroids for initialization...
Centroids calculated.


Using cache found in /root/.cache/torch/hub/facebookresearch_dino_main


artifact membership 'baseline-Davide-collab-test-checkpoints:latest' not found in 'aml-fl-project/fl-task-arithmetic'
Model checkpoint not found on WandB. artifact membership 'baseline-Davide-collab-test-checkpoints:latest' not found in 'aml-fl-project/fl-task-arithmetic'
Starting from scratch


Epoch 1/10: 100%|██████████| 782/782 [08:11<00:00,  1.59it/s, loss=1.19]


Epoch 1 Results: Train Loss: 1.7899 | Test Loss: 1.0647 | Test Acc: 68.96%
Model saved to WandB as artifact 'baseline-Davide-collab-test-checkpoints'.
0 Saved checkpoint model to WandB.


...

Epoch 7/10: 100%|██████████| 782/782 [08:10<00:00,  1.59it/s, loss=0.0167]


Epoch 7 Results: Train Loss: 0.0296 | Test Loss: 0.5356 | Test Acc: 85.42%
Model saved to WandB as artifact 'baseline-Davide-collab-test-checkpoints'.
6 Saved checkpoint model to WandB.


Epoch 8/10: 100%|██████████| 782/782 [08:11<00:00,  1.59it/s, loss=0.00712]


Epoch 8 Results: Train Loss: 0.0142 | Test Loss: 0.5436 | Test Acc: 85.78%
Model saved to WandB as artifact 'baseline-Davide-collab-test-checkpoints'.
7 Saved checkpoint model to WandB.


Epoch 9/10:  94%|█████████▍| 736/782 [07:42<00:28,  1.60it/s, loss=0.00573]